# Tutorial on using Propnet

The following is a tutorial designed to give a base overview of the classes and constructs used in the Propnet project. For each class an example of its construction and base usage is provided.

# Defining a Property Network: Propnet

A Graph object tells us about all the property types and models currently available for use.

The mappings contained in this object define an interconnected network of materials properties. In this form, the Graph object can be used to enumerate and analyze links between differnet materials properties.

In [1]:
from propnet.core.graph import Graph

Propnet is not intended for public use at this time. Functionality might change.



In [2]:
g = Graph()

You can print Propnet to see the property types and models it supports.

In [3]:
print(g)

Propnet Printout

Properties
	formula
	dimensionality
	structure_oxi
	oxide_type
	is_metallic
	magnetic_order
	lattice
	external_identifier_mp
	computed_entry
	external_identifier_aflow
	structure
	potentially_ferroelectric
	prototype
	composition
	unit_cell_heat_capacity_constant_volume
	atomic_density
	elastic_tensor_voigt
	gruneisen_parameter
	interplanar_spacing
	sound_velocity_longitudinal
	goldschmidt_tolerance_factor
	phonon_mean_free_path
	thermal_conductivity
	refractive_index
	piezoelectric_modulus_longitudinal_max_direction
	electron_concentration
	ionic_radius_b
	absorption_coefficient
	p_wave_modulus
	cost_per_kg
	extinction_coefficient
	total_magnetization_per_volume
	electronic_thermal_conductivity
	pilling_bedworth_ratio
	transmittance
	interatomic_spacing
	band_gap_pbe
	electrical_resistivity
	relative_permittivity
	snyder_acoustic_sound_velocity
	cost_per_mol
	sound_velocity_mean
	volume_unit_cell
	youngs_modulus
	dielectric_figure_of_merit_energy
	mass_per_atom
	pois

# Defining a Material Property

A Symbol object is used to represent types of properties (such as Young's Modulus) or conditions (such as Temperature).
- All Symbol objects are accessible in a global Registry entry.
- Various metadata for each SymbolType can be accessed as shown below.

In [4]:
# Import built-in symbols
import propnet.symbols
# Access them through symbol registry
from propnet.core.registry import Registry
symbol_object = Registry("symbols")['youngs_modulus']
print(symbol_object)

Symbol: youngs_modulus


A Quantity object is used to represent values of properties (such as Young's Modulus = 200GPa) or conditions (such as temperature = 300K).

- All Quantity objects have a Symbol giving the type of property represented by the value.
- All Quantity objects must be created at runtime by specifying a value during instantiation.
- All Quantity objects have a list of strings called "tags" used to further label the property.

Currently, two Quantity object types exist to support different kinds of data:

- NumQuantity supports numerical quantities, both scalar, vector, and matrix.
- ObjQuantity supports any JSON-serializable Python object (using MSONable). Useful for structure or formula.

Both types can be created using a QuantityFactory.

In [5]:
from propnet.core.quantity import QuantityFactory
steel_youngs_modulus = QuantityFactory.create_quantity('youngs_modulus', 200, 'GPa', tags=['mild steel'])
print(steel_youngs_modulus)
print(type(steel_youngs_modulus))

table_salt_formula = QuantityFactory.create_quantity('formula', 'NaCl', tags=['table salt'])
print(table_salt_formula)
print(type(table_salt_formula))

<youngs_modulus, 200 gigapascal, ['mild steel']>
<class 'propnet.core.quantity.NumQuantity'>
<formula, NaCl, ['table salt']>
<class 'propnet.core.quantity.ObjQuantity'>


# Defining a Material

A Material object is used to represent a collection of information known about a given material.

A Material object can be created empty and then populated with property data, or created from a list of properties.

In [6]:
from propnet.core.materials import Material
from propnet.core.quantity import NumQuantity, ObjQuantity
mild_steel = Material()
youngs_modulus = NumQuantity('youngs_modulus', 200, 'gigapascal')
mild_steel.add_quantity(youngs_modulus)
print(mild_steel)

table_salt_formula = ObjQuantity('formula', 'NaCl')
table_salt = Material([table_salt_formula])
print(table_salt)

Material: 0x127005898

	youngs_modulus
		<youngs_modulus, 200 gigapascal, []>

Material: 0x127005a58

	formula
		<formula, NaCl, []>



# Combining Models, Materials, and Symbols

As illustrated, a Graph object contains information for connecting many different models and symbol types. This forms an abstract web of interconnected variables without any values specified.

On the other hand, a Material object represents a grouping of values for different variables. These are represented as a collection of Quantity objects identified with the material.

At runtime, a single Graph object can be used to "evaluate" a Material object. This procedure allows values to be plugged in to variables. Assuming the required inputs for a model all have values, the Graph object can then dynamically predict the values for the output variables of the model.

In this example, we start with a refractive index and a permittivity. As a result many other properties are derived and printed below.

In [7]:
from propnet.core.materials import Material
from propnet.core.graph import Graph
from propnet.core.quantity import QuantityFactory
# Import built-in propnet models
import propnet.models

g = Graph()

silica = Material()
refractive_index = QuantityFactory.create_quantity('refractive_index', 1.458)
relative_permittivity = QuantityFactory.create_quantity('relative_permittivity', 3.9)

silica.add_quantity(refractive_index)
silica.add_quantity(relative_permittivity)

silica = g.evaluate(silica)
print(silica)

Material: 0x126af8cc0

	refractive_index
		<refractive_index, 1.458 dimensionless, []>

	relative_permittivity
		<relative_permittivity, 3.9 dimensionless, []>

	band_gap
		<band_gap, 21.02294283374089 electron_volt, []>
		<band_gap, 34.44429680416945 electron_volt, []>
		<band_gap, 8.44705399783503 electron_volt, []>

	relative_permeability
		...

	dielectric_figure_of_merit_current_leakage
		...
		...
		...

	band_gap_gw
		<band_gap_gw, 21.05104492358802 electron_volt, []>
		<band_gap_gw, 34.49929539495929 electron_volt, []>
		<band_gap_gw, 8.449953905646305 electron_volt, []>

	band_gap_pbe
		...
		...
		...
		...
		...
		...

	dielectric_figure_of_merit_energy
		...
		...
		...

	is_metallic
		<is_metallic, False, []>
		<is_metallic, False, []>
		<is_metallic, False, []>



# Working with Models

A Model object is used to represent a relationship between different materials property variables. This object can be directly manipulated and stores relavent metadata available as direct attributes.

- All Models are imported as classes at runtime.
- A Model class must be instantiated to be used at runtime.

In [8]:
import propnet.models
from propnet.core.graph import Graph
g = Graph()
# This information can be obtained from Registry("models") as well
model = g.get_models()['refractive_indexfrom_rel_perm']
print(model.description)
print(model.name)
print()
print(model.equations)

The refractive index gives the factor by which the speed of light is reduced in
a medium.  Likewise, modeling the induced magnetic and electric dipoles as linear
within a material, a relative spatial electrical permittivity and relative spatial
magnetic permeability arise from consideration of the total electrical and magnetic
fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean
of the relative permittivity and the relative permeability.

refractive_indexfrom_rel_perm

['n = sqrt(Ur*Er)']


## Model Class Structure - Basic Overview

The Model class is a generally-defined interface, and subclasses may alter many aspects of its underlying functionality.


Most Model objects will contain equations, symbols, and connections attributes. These define the core functionality of the model:

The equations attribute will contain a list of sympy-parsable expressions. These expressions imply trivial equations such that the expression is equal to zero. 
(ie. x - 4 for corresponding equation x = 4)

In [9]:
print(model.equations)

['n = sqrt(Ur*Er)']


The symbol_mapping attribute maps the symbols (ie. n) used in the equations to Symbol objects (ie. refractive index) used in the Property Network.

In [10]:
print(model.variable_symbol_map)

{'Ur': 'relative_permittivity', 'n': 'refractive_index', 'Er': 'relative_permeability'}


The connections attribute shows what outputs can be generated from a set of inputs, following a standard format. It also contains metadata for evaluating the models using equations.

In [11]:
print(model.connections)

[{'inputs': ['n', 'Er'], 'outputs': ['Ur'], '_sympy_exprs': {'Ur': [n**2/Er]}, '_lambdas': {'Ur': <function _lambdifygenerated at 0x1276bdae8>}}, {'inputs': ['Ur', 'Er'], 'outputs': ['n'], '_sympy_exprs': {'n': [sqrt(Er*Ur)]}, '_lambdas': {'n': <function _lambdifygenerated at 0x1276bd268>}}, {'inputs': ['Ur', 'n'], 'outputs': ['Er'], '_sympy_exprs': {'Er': [n**2/Ur]}, '_lambdas': {'Er': <function _lambdifygenerated at 0x1276bd7b8>}}]


A Model can be evaluated to generate outputs if given a complete set of inputs.

Given the relative permeability and permittivity, the Refractive Index From Relative Permeability model can correctly calculate the index of refraction.

This is given by 'n' in the dictionary below.

In [12]:
model.plug_in({'Ur': 0.54, 'Er': 3.9})

{'n': 1.4512063946937388}

# Loading Materials Data

Material properties can be loaded in from the Materials Project so they don't need to be defined and added manually.

Accessing Materials Project data requires an API key. You must input your own API key below to run the sample. You can locate your api key by logging into materialsproject.org and visiting the dashboard.

In [13]:
from propnet.ext.matproj import MPRester
# If you have a MP API key, you can enter it below.
my_api_key = None
mpr = MPRester(my_api_key)
silica = mpr.get_quantities_for_mpid('mp-546794')
print(silica)

{'material_id': 'mp-546794', 'band_gap.search_gap.band_gap': 5.703399999999999, 'computed_entry': ComputedEntry mp-546794 - Si16 O32
Energy = -379.9357
Correction = -22.4733
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Si', 'O'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Si', 'PBE O']
oxide_type = oxide
Data:
oxide_type = oxide, 'diel.n': 1.4782042033043565, 'diel.poly_total': 3.8824606666666672, 'diel.e_total': [[3.827947000000001, 0.0, 0.0], [0.0, 3.827947000000001, 1.9736472213591742e-17], [0.0, 1.9736472213591742e-17, 3.9914880000000004]], 'diel.poly_electronic': 2.1850876666666674, 'pretty_formula': 'SiO2', 'e_above_hull': 0, 'elasticity.elastic_tensor_original': [[91.20978053601698, -37.72413000560153, 19.319911320055816, 0.0, 0.0, 0.0], [-37.92987903108265, 91.35007557673609, 19.271159227260735, 0.0, 0.0, 0.0], [15.125369111160174, 15.125369111160174, 85.61256491670275, 0.0, 0.0, 0.0], [0.1477369604481241, 0.587

# Working with Units

## Symbol Objects

It is best practice to specify units when defining a Quantity object or plugging values into Models. However, in the case that no units are supplied, default units are inferred based on those provided in the corresponding Symbol object.

For instance, youngs_modulus is a Symbol with the default units of gigapascal. 
In the example above:

           youngs_modulus = Symbol('youngs_modulus', 200, [])
           
youngs_modulus automatically has the value 200 gigapascals despite no units being provided.

It is possible to provide your own units by working with the <b>unit registry</b> (ureg).<br></br>
The examples below demonstrate this capability using the method: 
            
            ureg.Quantity(value, unit)
            
propnet will automatically convert the units you supply into canonical units behind the scenes for use in models that use the property.

In [14]:
from propnet import ureg
from propnet.core.materials import Material
from propnet.core.quantity import QuantityFactory

mild_steel = Material()
yield_strength = QuantityFactory.create_quantity('yield_stress', ureg.Quantity(53700, 'psi'), [])
mild_steel.add_quantity(yield_strength)
print(mild_steel)

Material: 0x126d60320

	yield_stress
		<yield_stress, 0.3702484666431411 gigapascal, []>



## Models
Models can be evaluated using two methods: `plug_in` or `evaluate`.

The difference: `evaluate` is symbols in, symbols out; `plug_in` is variables in, variables out.

`evaluate` requires a dictionary keyed by symbol name (e.g. "youngs_modulus") and values which are propnet Quantity objects. Units are converted to canonical units on the fly.
`plug_in` requires a dictionary keyed by variable name (e.g. "Y") and values which are Python floats. No unit conversions are done.

In the example below we use propnet to calculate the Peierls-Nabarro Stress for a disloaction on the (111) plane in Aluminum (FCC) using custom units of psi for shear modulus.

In [15]:
from propnet import ureg
import propnet.models
from propnet.core.registry import Registry
from propnet.core.quantity import QuantityFactory

model = Registry("models")['peierls_stress']

result = model.evaluate({
    'shear_modulus': QuantityFactory.create_quantity("shear_modulus", 3.9e6, 'psi'),
    'interplanar_spacing': QuantityFactory.create_quantity("interplanar_spacing", 4.046 / 3**(1/2)),
    'interatomic_spacing': QuantityFactory.create_quantity("interatomic_spacing", 4.046 / 2**(1/2)),
    'poisson_ratio': QuantityFactory.create_quantity("poisson_ratio", 0.33)
})
print(result)

# Must use gigapascal, because model assumes gigapascal. See section below for conversions
G = 26.8896
a = 4.046 / 3**(1/2)  # Angstroms automatically assumed by the model.
b = 4.046 / 2**(1/2)  # Angstroms automatically assumed by the model.
nu = 0.33

tau = model.plug_in({'G': G, 'a': a, 'b': b, 'nu': nu})
print(tau)

{'peierls_stress': <peierls_stress, 0.012711228013154446 gigapascal, []>, 'successful': True}
{'T_pn': 0.01271125002140798}


## Converting Units
Given you have an output with units, it is straightforward to convert between units using Pint (the backend package used by propnet for units).

The `to` operation can be performed on propnet NumQuantity objects or pint ureg.Quantity objects.

In [16]:
# Converting to megapascals.
print(result['peierls_stress'].to('megapascal'))

<peierls_stress, 12.711228013154447 megapascal, []>


# Creating Custom Models and Properties

The property network comes with many different models and properties pre-loaded and ready for use.

To add additional properties to the project can be accomplished in two different forms:

- Defining additional .yaml files (symbols/models) or .py files (models) in the propnet.models or propnet.symbols folders. For these changes to take effect, propnet must be re-loaded.

- Defining additional Model (EquationModel, PyModel, etc.) and Symbol classes at runtime.

Both approaches will be detailed below.

## Defining new model files:
Depending on the nature of the model, it may be prudent choose a serialized yaml file or a Python module to define the new model. YAML files are used for simple models which can be represented by a single equation and generally only deal with scalar quantities. Python module models are for more complex models which involve matrices/vectors or more complex evaluation.

The syntax for an equation-based model stored in a yaml file is as follows:

<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Required?</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>name</td>
    <td>yes</td>
    <td>string</td> 
    <td>name of the model, must match the file name, no spaces</td>
  </tr>
  <tr>
    <td>categories</td>
    <td>yes, but may be empty</td>
    <td>list of strings</td> 
    <td>any user-specific tags applying to the model</td>
  </tr>
  <tr>
    <td>description</td>
    <td>yes, but may be empty</td>
    <td>string</td> 
    <td>ideally, a verbose description of the model, its assumptions, etc.</td>
  </tr>
  <tr>
    <td>implemented_by</td>
    <td>yes</td>
    <td>list of strings</td> 
    <td>GitHub usernames of author(s) of the model. May not be empty.</td>
  </tr>
  <tr>
    <td>references</td>
    <td>no, but recommended</td>
    <td>list of strings</td> 
    <td>url, doi, or isbn for citing the model, use "type:data" to specify, e.g. "url:https://www...."</td>
  </tr>
  <tr>
    <td>equations</td>
    <td>yes</td>
    <td>list of strings</td> 
    <td>list of equations that create the model</td>
  </tr>
  <tr>
    <td>variable_symbol_map</td>
    <td>no, but see description</td>
    <td>dictionary, string keys, string values</td> 
    <td>mapping from variables used in the equation to symbol names used in propnet, e.g. {"m": "mass"}. If not specified, the variable names will be assumed to be the same as the symbol names.</td>
  </tr>
  <tr>
    <td>units_for_evaluation</td>
    <td>no, but see description</td>
    <td>dictionary, string keys, string values</td> 
    <td>mapping from variables used in the equation to units required for proper evaluation. It is required if the model is empirical or includes constants that have dimensions.</td>
  </tr>
  <tr>
    <td>connections</td>
    <td>no, but see description</td>
    <td>list of dictionaries with keys: 'inputs', 'outputs' mapping to lists of symbol strings used in the model</td> 
    <td>contains valid input/output symbol combinations. If omitted, this will be derived automatically from the equation. For example,
    F=m*a will produce {"inputs": ["mass", "acceleration"], "outputs": ["force"]} and no other combination.</td>
  </tr>
  <tr>
    <td>solve_for_all_symbols</td>
    <td>no, default false</td>
    <td>boolean</td> 
    <td>If true, propnet will attempt to derive all possible input/output combinations from the equation. Default is false.</td>
  </tr>
  <tr>
    <td>constraints</td>
    <td>no</td>
    <td>list of strings</td>
    <td>(in)equality statements for input or output variables that must be true in order for the model to be evaluated successfully. e.g. "Eg > 0"</td>
  </tr>
  <tr>
    <td>test_data</td>
    <td>no, but recommended</td>
    <td>list of dictionaries with keys: 'inputs', 'outputs', which have dictionaries, keyed by variables, valued by numerical or string input(s) and expected numerical or string output(s)</td>
    <td>data used to test the model. Data may be specified as raw numbers, assumed to be the unit specified by units_for_evaluation or by the default symbol unit. Data may also be specified as strings of numbers and units. Example: [{'inputs': {'m': '3 kg', 'a': '2 m/s**2'}, 'outputs': {'F': '6 N'}}]</td>
  </tr>
</table>

Python models are constructed in a very similar manner to the YAML file. To create a new Python model:

1. Create a new py module in the models/python folder with the name of your model as the file name.
2. In your module, define a function which takes a dictionary of inputs, keyed by input variable name, and returns a dictionary, keyed by output variable name. This is your "plug-in" function.
3. Define a module-level variable `config` which holds a dictionary with the fields specified in the YAML construction above. The only exceptions are that `connections` must be explicitly included, `solve_for_all_symbols` and `equations` cannot be used, and `config` must have a field `plug_in` which is valued by a reference to your plug-in function from step 2.
4. If your model requires Python packages not already required by propnet, be sure to add them to "requirements.txt" under the "# specific models" section.


## Defining new Symbol files:
Symbols require yaml file to be defined in the propnet/symbols folder. For numerical properties which aren't environmental conditions, place them in the "properties" folder. For properties which do not hold numbers (e.g. formula), place them in the "objects" folder.

The syntax for a yaml definition is as follows:

<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Required?</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>name</td>
    <td>yes</td>
    <td>string</td> 
    <td>name of the property, must match the file name</td>
  </tr>
  <tr>
    <td>category</td>
    <td>yes</td>
    <td>must be one of ('property', 'condition', 'object')</td> 
    <td>specifies the type of data the symbol represents</td>
  </tr>
  <tr>
    <td>units</td>
    <td>depends, see decription</td>
    <td>string</td> 
    <td>the units used to represent this symbol, must be specified for property and condition symbols, must be excluded for object symbols.</td>
  </tr>
  <tr>
    <td>display_names</td>
    <td>no, but recommended</td>
    <td>list of strings</td> 
    <td>descriptive name(s) that represent this symbol</td>
  </tr>
  <tr>
    <td>display_symbols</td>
    <td>no, but recommended</td>
    <td>list of strings</td> 
    <td>any short string symbols useful for representing this symbol, for example, how it would be represented in an equation</td>
  </tr>
  <tr>
    <td>shape</td>
    <td>depends, see description</td>
    <td>number or list of numbers</td>
    <td>expected dimesions of a numerical symbol. 1 or [1]=scalar, n or [n]=vector of length n, [m, n]=matrix of m-by-n size. required for properties and conditions, must be excluded for objects.</td>
  </tr>
  <tr>
    <td>constraint</td>
    <td>no</td>
    <td>string</td> 
    <td>string representing a constraint equation required for the symbol, e.g. 'bulk_modulus > 0'. Only applicable to property and condition symbols.</td>
  </tr>
  <tr>
    <td>object_type</td>
    <td>no</td>
    <td>string</td> 
    <td>string representing the Python object type expected by an object symbol. e.g. "str" or "pymatgen.core.lattice.Lattice". Only applicable to object symbols. If specified, quantities created with this symbol will be cast as this type.</td>
  </tr>
  <tr>
    <td>comment</td>
    <td>no</td>
    <td>string</td> 
    <td>any comments pertinent to the symbol</td>
  </tr>
  <tr>
    <td>default_value</td>
    <td>no</td>
    <td>float</td> 
    <td>default value for a quantity made from this symbol, generally used for conditions (may be deprecated)</td>
  </tr>
</table>

# A Grand Example

In [17]:
silicon = Material()
silicon.add_quantity(QuantityFactory.create_quantity('temperature', 300, []))
silicon.add_quantity(QuantityFactory.create_quantity('band_gap', 1.12, []))
silicon.add_quantity(QuantityFactory.create_quantity('electrical_conductivity', 4.34E-4, []))

aluminum = Material()
aluminum.add_quantity(QuantityFactory.create_quantity('temperature', 300, []))
aluminum.add_quantity(QuantityFactory.create_quantity('band_gap', 0, []))
aluminum.add_quantity(QuantityFactory.create_quantity('electrical_conductivity', 3.69E7, []))

g = Graph()
silicon = g.evaluate(silicon)
aluminum = g.evaluate(aluminum)

print("Silicon's Graph")
print(silicon)
print()
print("Aluminum's Graph")
print(aluminum)

Silicon's Graph
Material: 0x127e4ad68

	temperature
		<temperature, 300 kelvin, []>

	band_gap
		<band_gap, 1.12 electron_volt, []>

	electrical_conductivity
		...

	refractive_index
		<refractive_index, 3.194112 dimensionless, []>
		...
		...
		...
		...

	band_gap_gw
		...

	band_gap_pbe
		...
		...

	is_metallic
		<is_metallic, False, []>

	electronic_thermal_conductivity
		...

	electrical_resistivity
		...


Aluminum's Graph
Material: 0x127005a90

	temperature
		<temperature, 300 kelvin, []>

	band_gap
		<band_gap, 0 electron_volt, []>

	electrical_conductivity
		...

	refractive_index
		<refractive_index, 4.16 dimensionless, []>
		...

	band_gap_gw
		...

	is_metallic
		<is_metallic, True, []>

	electronic_thermal_conductivity
		...

	electrical_resistivity
		...

